<a href="https://colab.research.google.com/github/olcaykursun/ML/blob/main/neuralnets/Transfer_Learning_with_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Transfer Learning Implementation and Usage with CNNs</b>  
AUM Machine Learning, Dr. Olcay Kursun, okursun@aum.edu  
Date: 04/02/2024 (Spring 2024)  

Description: This script demonstrates the implementation of a neural network model using TensorFlow and Keras. It covers the construction, compilation, and training of the model, along with examples of transfer learning and feature extraction.


In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Load the full dataset
(full_x_train, full_y_train), (full_x_test, full_y_test) = fashion_mnist.load_data()

# Normalize the pixel values
full_x_train, full_x_test = full_x_train / 255.0, full_x_test / 255.0

full_x_train.shape

(60000, 28, 28)

In [ ]:
# Expand dimensions to include channel information
full_x_train = np.expand_dims(full_x_train, -1)
full_x_test = np.expand_dims(full_x_test, -1)

full_x_train.shape

(60000, 28, 28, 1)

In [ ]:
# Let's say we keep the first n1 classes, build our features, and transfer that knowledge of how to extract features
# to solve a different problem: A problem with few examples that belong to a totally new set of classes
# for example if n1=8 then we remove classes 8 and 9 for this example and use classes 0-7 for learning good features
n1 = 6        # We will use all the examples of these n1 classes: base task is to learn the base model
n2 = 10 - n1  # We will use only 5 examples: target task is uses base to solve this new problem with a small dataset

base_classes = range(n1)
idx_train = np.isin(full_y_train, base_classes)
idx_test = np.isin(full_y_test, base_classes)

base_x_train, base_y_train = full_x_train[idx_train], full_y_train[idx_train]
base_x_test, base_y_test = full_x_test[idx_test], full_y_test[idx_test]

# Convert labels to one-hot encoding
base_y_train = to_categorical(base_y_train, num_classes=n1)
base_y_test = to_categorical(base_y_test, num_classes=n1)

base_y_train.shape

(36000, 6)

In [ ]:
# We will test the quality of the features learned on the n2 classes with few examples
num_samples_per_class = 5

target_classes = range(n1, 10)
idx_train = np.isin(full_y_train, target_classes)
idx_test = np.isin(full_y_test, target_classes)

target_x_train, target_y_train = full_x_train[idx_train], full_y_train[idx_train]

all_selected_indices = []
for class_label in target_classes:
    # Indices of all instances of the current class
    class_indices = np.where(target_y_train == class_label)[0]

    # Randomly choose num_samples_per_class indices from this class
    chosen_indices = np.random.choice(class_indices, num_samples_per_class, replace=False)

    # Append these indices to the list
    all_selected_indices.extend(chosen_indices)  # Use extend or +

# Use the selected indices to build the small dataset for the target task
target_x_train, target_y_train = target_x_train[all_selected_indices], target_y_train[all_selected_indices]

target_x_test, target_y_test = full_x_test[idx_test], full_y_test[idx_test]

# Convert labels to one-hot encoding
target_y_train = to_categorical(target_y_train - n1, num_classes=n2)
target_y_test = to_categorical(target_y_test - n1, num_classes=n2)
# For binary (class 8 vs 9), we could also use:
#target_y_train = 0+(target_y_train == 8)   #also works: (target_y_train == 8).astype(int)

target_y_train.shape

(20, 4)

In [ ]:
idx_train

array([ True, False, False, ..., False, False, False])

In [ ]:
target_x_train.shape

(20, 28, 28, 1)

In [ ]:
from tensorflow.keras import layers, models

input_shape = (28, 28, 1)

model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n1, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2024-04-03 03:16:51.266975: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-03 03:16:51.266996: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-03 03:16:51.267002: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-03 03:16:51.267032: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-03 03:16:51.267045: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [ ]:
# Train the model
model.fit(base_x_train, base_y_train, epochs=5, validation_split=0.2)

# Evaluate the new model on the test set
print('\nNot the focus of transfer learning, but a good accuracy here shows learned base features can be good:')
test_loss, test_acc = model.evaluate(base_x_test, base_y_test)
print(f"Base-task test loss: {test_loss}")
print(f"Base-task test accuracy: {test_acc}")

# Alternative to "evaluate", we can use the model to predict the classes of the test set and then calculate the accuracy
predictions = model.predict(base_x_test)
# The predictions are in the form of probabilities for each class.
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(base_y_test, axis=1)
# Calculate accuracy by comparing predicted and true classes
accuracy = np.mean(predicted_classes == true_classes)
print(f"Alternative way of computing accuracy: {accuracy}")

Epoch 1/5
  1/900 [..............................] - ETA: 4:20 - loss: 1.7915 - accuracy: 0.2188

2024-04-03 03:16:51.639028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


900/900 [==============================] - 9s 10ms/step - loss: 0.3390 - accuracy: 0.8756 - val_loss: 0.2331 - val_accuracy: 0.9196
Epoch 2/5
900/900 [==============================] - 9s 10ms/step - loss: 0.2188 - accuracy: 0.9225 - val_loss: 0.2126 - val_accuracy: 0.9239
Epoch 3/5
900/900 [==============================] - 9s 10ms/step - loss: 0.1877 - accuracy: 0.9346 - val_loss: 0.2066 - val_accuracy: 0.9265
Epoch 4/5
900/900 [==============================] - 9s 10ms/step - loss: 0.1723 - accuracy: 0.9402 - val_loss: 0.2001 - val_accuracy: 0.9324
Epoch 5/5
900/900 [==============================] - 9s 10ms/step - loss: 0.1570 - accuracy: 0.9456 - val_loss: 0.1873 - val_accuracy: 0.9369

Not the focus of transfer learning, but a good accuracy here shows learned base features can be good:
188/188 [==============================] - 1s 7ms/step - loss: 0.1806 - accuracy: 0.9392
Base-task test loss: 0.18056409060955048
Base-task test accuracy: 0.9391666650772095
188/188 [==============

In [ ]:
# Extract features
# We can feed them to Decision Tree (instead of the dense layers forming an MLP in the original setup)
# We can feed them to a clustering algorithm as inputs for clustering never before seen examples and classes

# remove the MLP from the top (the two dense layers)
feature_extractor = models.Model(inputs=model.input, outputs=model.layers[-3].output)
features_target_train = feature_extractor.predict(target_x_train) #get features for the training set of the target task
features_target_test = feature_extractor.predict(target_x_test) #get features for the test set of the target task

125/125 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Train the decision tree using the features learned from the base task
tree = DecisionTreeClassifier()
tree.fit(features_target_train, np.argmax(target_y_train, axis=1))  # Fit to the max class index

# Evaluate the decision tree
accuracy = tree.score(features_target_test, np.argmax(target_y_test, axis=1))
print(f"Target-task Decision Tree Accuracy (using features learned from the base): {accuracy}")


Target-task Decision Tree Accuracy (using features learned from the base): 0.7375


In [ ]:
# Use the raw features for training/testing the decision tree

num_input_channels = np.prod(input_shape)

# Instantiate a new decision tree and train on the small dataset without knowledge-transfer (from scratch)
tree = DecisionTreeClassifier()
tree.fit(target_x_train.reshape(-1, num_input_channels), np.argmax(target_y_train, axis=1))  # Fit to the max class index

# Evaluate the decision tree
accuracy = tree.score(target_x_test.reshape(-1, num_input_channels), np.argmax(target_y_test, axis=1))
print(f"Target-task Decision Tree Accuracy using raw features: {accuracy}")


Target-task Decision Tree Accuracy using raw features: 0.6865


In [ ]:
# Learned features can be used as input to a standalone MLP as well

# New model using the extracted features for n2-class classification problem
new_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(features_target_train.shape[1],)),
    layers.Dense(n2, activation='softmax')  #For n2=1 can also be replaced by (1, 'sigmoid') and binary_crossentropy
])

# Train the new model on the target-task
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(features_target_train, target_y_train, epochs=5)

# Evaluate the new model on the test set
test_loss, test_acc = new_model.evaluate(features_target_test, target_y_test)

print(f"Standalone MLP Test loss: {test_loss}")
print(f"Standalone MLP Test accuracy: {test_acc}")


Epoch 1/5
1/1 [==============================] - 0s 249ms/step - loss: 1.4517 - accuracy: 0.3000
Epoch 2/5
1/1 [==============================] - 0s 12ms/step - loss: 1.1980 - accuracy: 0.6500
Epoch 3/5
1/1 [==============================] - 0s 14ms/step - loss: 0.9948 - accuracy: 0.8000
Epoch 4/5
1/1 [==============================] - 0s 11ms/step - loss: 0.8262 - accuracy: 0.9500
Epoch 5/5
125/125 [==============================] - 1s 7ms/step - loss: 0.7997 - accuracy: 0.8202
Standalone MLP Test loss: 0.7997000813484192
Standalone MLP Test accuracy: 0.8202499747276306


In [ ]:
# Start fine-tuning

# Create a new classifier-layer on top use unpacking of the layers of the model, shallow copied

# This will perform shallow copy of the base layers
# So, if there were complex connections in the base, then this should be used as it preserves the architecture
# as opposed to the unpacking method below

from tensorflow.keras import models, layers

# In Keras, layers and models (including submodels created with the Model constructor) are all callable objects.
# This means you can treat them like functions and pass input tensors to them to obtain output tensors.
truncated_pretrained_model = models.Model(inputs=model.inputs, outputs=model.layers[-3].output)

target_model = models.Sequential([
    truncated_pretrained_model,         # Use the submodel/functional at first, again a form of shallow copy of the layers
    layers.Dense(64, activation='relu'),
    layers.Dense(n2, activation='softmax')
])

# Freeze all layers coming from the base model
for layer in target_model.layers[:-2]:
    layer.trainable = False

# Compile the model
target_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target_model.fit(target_x_train, target_y_train, epochs=5)
# Evaluate the target_model on the test set
test_loss, test_acc = target_model.evaluate(target_x_test, target_y_test)

print(f"target_model test loss: {test_loss}")
print(f"target_model test accuracy: {test_acc}")


Epoch 1/5
1/1 [==============================] - 0s 179ms/step - loss: 1.4541 - accuracy: 0.2500
Epoch 2/5
1/1 [==============================] - 0s 11ms/step - loss: 1.1846 - accuracy: 0.6500
Epoch 3/5
1/1 [==============================] - 0s 12ms/step - loss: 0.9717 - accuracy: 0.7500
Epoch 4/5
1/1 [==============================] - 0s 17ms/step - loss: 0.7862 - accuracy: 0.9500
Epoch 5/5
125/125 [==============================] - 1s 7ms/step - loss: 0.7466 - accuracy: 0.8570
target_model test loss: 0.7466206550598145
target_model test accuracy: 0.8569999933242798


In [ ]:
# Unfreeze all layers in the target_model, not recommended for small datasets
# Due to the shallow copy above, the base model will also change
for layer in target_model.layers:
    layer.trainable = True

# Compile the model, because it will initialize the computation graph that contains learning rates, momentums etc.
target_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target_model.fit(target_x_train, target_y_train, epochs=5)

# Evaluate the target_model on the test set
test_loss, test_acc = target_model.evaluate(target_x_test, target_y_test)

print(f"Retrained target_model test loss: {test_loss}")
print(f"Retrained target_model test accuracy: {test_acc}")

print(target_model.layers[0]==model.layers[0])
weights_model = model.layers[0].get_weights()
weights_target_model = target_model.layers[0].get_weights()
if np.array_equal(weights_model[1], weights_target_model[1]):
    print("Weights of the first layers are the same.")
else:
    print("Weights of the first layers are different.")


Epoch 1/5
1/1 [==============================] - 0s 232ms/step - loss: 0.4986 - accuracy: 0.9500
Epoch 2/5
1/1 [==============================] - 0s 15ms/step - loss: 0.3518 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 15ms/step - loss: 0.2392 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 13ms/step - loss: 0.1555 - accuracy: 1.0000
Epoch 5/5
125/125 [==============================] - 1s 7ms/step - loss: 0.3539 - accuracy: 0.9020
Retrained target_model test loss: 0.35391634702682495
Retrained target_model test accuracy: 0.9020000100135803
False
Weights of the first layers are the same.


In [ ]:
# Unpacking approach is not preferred

target_model = models.Sequential([
    *model.layers[:-2],
    layers.Dense(64, activation='relu'),
    layers.Dense(n2, activation='softmax')
])

# Freeze all layers coming from the base model
for layer in target_model.layers[:-2]:
    layer.trainable = False

# Compile the model
target_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target_model.fit(target_x_train, target_y_train, epochs=5)
# Evaluate the target_model on the test set
test_loss, test_acc = target_model.evaluate(target_x_test, target_y_test)

print(f"target_model test loss: {test_loss}")
print(f"target_model test accuracy: {test_acc}")


Epoch 1/5
1/1 [==============================] - 0s 198ms/step - loss: 1.3599 - accuracy: 0.4000
Epoch 2/5
1/1 [==============================] - 0s 14ms/step - loss: 1.0367 - accuracy: 0.6000
Epoch 3/5
1/1 [==============================] - 0s 13ms/step - loss: 0.8118 - accuracy: 0.8500
Epoch 4/5
1/1 [==============================] - 0s 14ms/step - loss: 0.6183 - accuracy: 0.9000
Epoch 5/5
125/125 [==============================] - 1s 7ms/step - loss: 0.5887 - accuracy: 0.8665
target_model test loss: 0.5886551141738892
target_model test accuracy: 0.8665000200271606


In [ ]:
# Copy options: saving/loading the model, cloning, and serialization/deserialization.

from tensorflow.keras.models import clone_model, Model
from tensorflow.keras.layers import Input, Dense

cloned_model = clone_model(model)
cloned_model.set_weights(model.get_weights())

model_input = Input(shape=input_shape)
x = model_input
for layer in cloned_model.layers[:-2]:
    x = layer(x)
x = Dense(64, activation='relu')(x)
new_output = Dense(n2, activation='softmax')(x)
target_model = Model(inputs=model_input, outputs=new_output)

for layer in target_model.layers[:-2]:
    layer.trainable = False

target_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target_model.fit(target_x_train, target_y_train, epochs=5, verbose=0)
test_loss, test_acc = target_model.evaluate(target_x_test, target_y_test)
print(f"target_model test loss: {test_loss}")
print(f"target_model test accuracy: {test_acc}")


125/125 [==============================] - 1s 7ms/step - loss: 0.6410 - accuracy: 0.8568
target_model test loss: 0.641007661819458
target_model test accuracy: 0.8567500114440918


In [ ]:
# In a way, cloning approaches work like this:
# Manually replicate the architecture of the original model and copy the weights of corresponding layers.

target_model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n2, activation='softmax')
])

for i, layer in enumerate(model.layers[:-2]):
    target_model.layers[i].set_weights(layer.get_weights())
    target_model.layers[i].trainable = False

target_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

target_model.fit(target_x_train, target_y_train, epochs=5, verbose=0)
test_loss, test_acc = target_model.evaluate(target_x_test, target_y_test)
print(f"target_model test loss: {test_loss}")
print(f"target_model test accuracy: {test_acc}")


125/125 [==============================] - 1s 7ms/step - loss: 0.5346 - accuracy: 0.8745
target_model test loss: 0.5345913767814636
target_model test accuracy: 0.8744999766349792
